In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
batch_size=64
lr=0.01
momentum=0.5
epochs=2
log_interval=10

In [3]:

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)



Processing...
Done!


In [4]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    
model = Net()

In [5]:

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


In [6]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
#         if args.cuda:
#             data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


In [7]:

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
#         if args.cuda:
#             data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [8]:

for epoch in range(1,epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327337
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.306440
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.251766
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.281436
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.265142
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.250026
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.173754
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.072551
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.142535
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.930265
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.992997
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.788385
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.640832
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.631914
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.372564
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.380614
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.452266
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.327140
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.062072
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.114332
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.314469
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.272589
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.656277
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.362533
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.424803
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.256191
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.299532
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.355502
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.312200
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.273123
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.286899
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.523901
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.328993
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.276344
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.413681
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.321205
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.280560
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.627433
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.343604
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.327446


In [18]:
w1 = model.parameters()
list(w1)

[Parameter containing:
 (0 ,0 ,.,.) = 
   0.2128  0.2661  0.2396 -0.1836 -0.3214
   0.0157  0.1445  0.2532 -0.0322 -0.2574
  -0.0198  0.0518  0.3674  0.1567 -0.0764
  -0.2078 -0.0753  0.3650  0.3314  0.3167
   0.0271  0.0753  0.2105  0.1834  0.0159
 
 (1 ,0 ,.,.) = 
   0.0091 -0.0887  0.2363 -0.0788 -0.0898
   0.1534  0.1306  0.2471  0.2930 -0.0520
  -0.1567 -0.1610  0.1837  0.1065  0.1585
   0.0767 -0.0614 -0.1659  0.1720  0.0434
  -0.3042 -0.2339 -0.0299 -0.2120  0.1300
 
 (2 ,0 ,.,.) = 
  -0.1439  0.1628  0.0856 -0.0226 -0.1260
  -0.1116 -0.0772  0.2109  0.0476 -0.2457
  -0.1226  0.0633  0.1716  0.1777 -0.2553
  -0.0661  0.0993  0.1912  0.2740  0.2219
   0.2091  0.2903  0.2456  0.4195  0.4125
 
 (3 ,0 ,.,.) = 
  -0.0737  0.0990 -0.2314 -0.1565  0.2563
  -0.1747 -0.0674 -0.2561  0.1521  0.2682
  -0.1640 -0.0754 -0.1645  0.1075  0.2138
   0.0290  0.0320 -0.1092 -0.0362  0.2153
  -0.1576 -0.0551 -0.0627  0.2720  0.0777
 
 (4 ,0 ,.,.) = 
   0.2749  0.1944 -0.1642 -0.1272 -0.1396
   0.21

In [19]:
torch.save(model,"./mnist.pt")

/home/aditya/anaconda3/lib/python3.6/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:
x=torch.load("./mnist.pt")

In [24]:
print(list(x.parameters()))

[Parameter containing:
(0 ,0 ,.,.) = 
  0.2128  0.2661  0.2396 -0.1836 -0.3214
  0.0157  0.1445  0.2532 -0.0322 -0.2574
 -0.0198  0.0518  0.3674  0.1567 -0.0764
 -0.2078 -0.0753  0.3650  0.3314  0.3167
  0.0271  0.0753  0.2105  0.1834  0.0159

(1 ,0 ,.,.) = 
  0.0091 -0.0887  0.2363 -0.0788 -0.0898
  0.1534  0.1306  0.2471  0.2930 -0.0520
 -0.1567 -0.1610  0.1837  0.1065  0.1585
  0.0767 -0.0614 -0.1659  0.1720  0.0434
 -0.3042 -0.2339 -0.0299 -0.2120  0.1300

(2 ,0 ,.,.) = 
 -0.1439  0.1628  0.0856 -0.0226 -0.1260
 -0.1116 -0.0772  0.2109  0.0476 -0.2457
 -0.1226  0.0633  0.1716  0.1777 -0.2553
 -0.0661  0.0993  0.1912  0.2740  0.2219
  0.2091  0.2903  0.2456  0.4195  0.4125

(3 ,0 ,.,.) = 
 -0.0737  0.0990 -0.2314 -0.1565  0.2563
 -0.1747 -0.0674 -0.2561  0.1521  0.2682
 -0.1640 -0.0754 -0.1645  0.1075  0.2138
  0.0290  0.0320 -0.1092 -0.0362  0.2153
 -0.1576 -0.0551 -0.0627  0.2720  0.0777

(4 ,0 ,.,.) = 
  0.2749  0.1944 -0.1642 -0.1272 -0.1396
  0.2183 -0.0365 -0.2747 -0.2110  0.1